# Data processing

The goal of this notebook is to process and manipulate the data in order to engineer features for exploration and modeling of the banker's offers. While the data is not completed structured in a tidy format based on the rules of the game (contestants may or may not take deals, the host may let the contestant continue selecting cases after a deal has been made, etc.), the relevant data points for this analysis will be at the game-round data. I will therefore manipulate the data to this level, as well as keeping track of both selected and remaining cases at the end of each round for each game in a list; while the data is no longer 'tidy' in the traditional sense (multiple values in a single cell), it permits me to aggregate the values within the cell at that point in the game more easily. My intuition from watching games suggests that there is a relationship between the remaining values on the board and the offer.

In [31]:
import pandas as pd
import numpy as np

In [32]:
games = pd.read_csv('games.csv')
case_values = pd.read_csv('case_values.csv')
contestants = pd.read_csv('contestants.csv')

In [ ]:
def create_lists_value(lst):
    '''Creates a list within the cell that includes all values selected up to that poin in the game'''
    return [list(lst[:i+1]) for i in range(len(lst))]

In [ ]:
games['filter_column'] = games.groupby(['ID', 'round'])['round_turn'].rank(method='first', ascending=False) == 1
games['selected_values'] = games.sort_values(['ID', 'round', 'round_turn']).groupby('ID')['value'].apply(create_lists_value).explode().reset_index()['value']
games['remaining_values'] = games.apply(lambda x: sorted(set(case_values_list) - set(x['selected_values'])), axis=1)

In [ ]:
original_case_values = games.query('original_case.notnull()')[['ID', 'value']].rename(columns={'value' :'original_case_value'})

In [ ]:
round_data = games.query('filter_column')[['ID', 'round', 'offer', 'decision_made', 'winnings', 'original_case', 'selected_values', 'remaining_values']]
round_data = round_data.merge(original_case_values, on='ID')

In [ ]:
## safely calculating values from lists embedded within cells
def safe_average(lst):
    if lst:
        try:
            value = sum(lst) / len(lst)
        except ZeroDivisionError:
            value = 0
    else:
        value = None
    return value

def safe_count_threshold(lst, threshold):
    if lst:
        value = len([x for x in lst if x >= threshold])
    else:
        value = None
    return value

def safe_prop_threshold(lst, threshold):
    if lst:
        try:
            value = len([x for x in lst if x >= threshold]) / len(lst)
        except ZeroDivisionError:
            value = 0
    else:
        value = None
    return value


In [ ]:
round_data['remaining_avg'] = round_data['remaining_values'].apply(safe_average)
round_data['remaining_count_right'] = round_data['remaining_values'].apply(safe_count_threshold, args=(1_000,))
round_data['remaining_count_big'] = round_data['remaining_values'].apply(safe_count_threshold, args=(100_000,))
round_data['remaining_prop_right'] = round_data['remaining_values'].apply(safe_prop_threshold, args=(1_000,))
round_data['remaining_prop_big'] = round_data['remaining_values'].apply(safe_prop_threshold, args=(100_000,))
round_data['offer_avg_ratio'] = round_data['offer'].div(round_data['remaining_avg'])

In [ ]:
## adding demographic data
round_data = round_data.merge(contestants[['ID', 'season', 'episode', 'gender', 'race']], on='ID')

In [ ]:
round_data.head()

In [ ]:
# round_data.to_csv('dond_processed_data.csv', index=False)